In [45]:
from ensemble import *
import itertools,operator

In [57]:
def inverdic(dic):
  resdic = {}
  for key, value in dic.items():
    for index in value:
      if type(value) == set or type(value) == list:
        if index in resdic.keys():
          if isinstance(resdic[index], set):
            resdic[index].add(key)
          elif isinstance(resdic[index], list):
            resdic[index].append(key)
          else:
            resdic[index] = [resdic[index]]
            resdic[index].append(key)
        else:
          resdic[index] = key
      elif type(value) == dict:
        resdic.update(inverdic(value))
  return resdic

In [563]:
"""
Словарь, который используется для детекции номера дома, корпуса и т.д.
"""
sep_house_signs = {
  'дом': {'д', 'дом'},
  'владение':{'владение', 'вл'},
  'корпус': {'к', 'корп', 'копр', 'кор', 'корпус'},
  'строение':{'с', 'стр', 'строен', 'строение'},
  'квартира': {'кв', 'квартира'},
  'помещение':{'пом', 'помещение'},
  'комната':{"ком",'комн', 'комната'},
  'кабинет':{"кабинет", "каб", "к-т", "каб-т"},
  'офис': {'оф', 'офис'},
  'литера': set("абвежз"),
  'прочее': {'литер', 'литера', 'лит'},
  'дробь':{'/', '-'}
}

house_signs_inv = inverdic(sep_house_signs)

In [685]:
def boost_keyword(dic):
  replaces_lowered = {}
  for key, value in replaces.items():
    if isinstance(key, tuple):
      new_key = "(" + ' OR '.join(key) + ')^'+str(1/len(key))
      replaces_lowered[new_key] = value
    else:
      replaces_lowered[key] = value
  return replaces_lowered

In [748]:
'''
Этот словарь приводит все типы адресных объектов к стандартному виду (к тому что в ФИАС)
'''

replaces = {
  'обл': {"область", "обл", "обл-ть"},
  'респ': {"республика", 'респ'},
  'край': {'край'},
  'г': {'г', 'гор', 'город'},
  ('ао', 'а.окр'):{'автономный округ', "автономный", 'аокр',  'а.окр'},
  ('а.обл', 'аобл'):{'автономная область', 'авт.обл', 'аобл', 'а обл', 'аобл'},

  ('аллея', 'ал'):{'аллея', 'а', 'ал'},
  'б-р':{'б-р', 'бульвар'},
  'наб':{'наб', 'набережная'},
  'пер':{'пер', 'переулок'},
  ('площадь', 'пл'):{'пл', "площадь"},
  ('проспект', 'пр-кт'):{"проспект", "пр", "пр-кт", "просп", 'пр-т'},
  "пр-д":{"проезд", "пр-д", "прд"},
  "ул":{"улица", "ул", "у", 'ул-ца'},

  'р-н':{'район', "р" , "р-н"},
  'п':{'поселок', 'посёлок', "пос"},
  'пгт':{'поселок городского типа', 'посёлок городского типа',  'пос. гор. типа', 'пос.гор.типа', 'пос гор типа'},

  'г':{'г', 'гор', 'город'},
  'с':{'с', 'село', 'сел'},
  'д':{'д', 'дер', 'деревня', 'д-ня'},
  'с/п':{"сельский поселок", "сельский посёлок", "сп", 'сельское поселение', 'сельпо', 'сп', 'сел.п.', }
}
replaces_inv = inverdic(boost_keyword(replaces))

In [212]:
def del_sp_char(string):
  '''
  Стоплист. Удаляет из строки символы переноса строки, но словарь можно дополнить при надобности
  '''
  for stopword in {r'\n', r'\r', '\\', '(', ')', ':'}:
    string = string.replace(stopword, ' ')
  return re.sub(r"[\d]+", ' \g<0> ', string)

def preprocess(string):
  string = del_sp_char(string)
  string = re.sub(r"[\d]+|[\W]+", ' \g<0> ', string)
  
  string = string.replace(',', ', ')
  string = re.sub(r'\,|\.|\-|\'|\"|\(|\)', '', string)
  string = re.sub(r' +', ' ', string)
  return string

In [163]:
def multiple_replace(dict, text, compiled=False): 

  """ Replace in 'text' all occurences of any key in the given
  dictionary by its corresponding value.  Returns the new tring.""" 

  
  # Create a regular expression  from the dictionary keys
  
  if not compiled:
    regex = re.compile(r"\b(%s)\b" % "|".join(map(re.escape, dict.keys())))
  else:
    regex = compiled

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

In [55]:
def tokenize(string, comma=False):
  '''
  Токенизатор. Раздвигает слипшиеся буквы и цифры вроде 2с3 или корп1
  Вход: строка
  Выход: массив из слов (токены)
  '''
  # string = re.sub(r"[\d]+", ' \g<0> ', string).lower() — это уже сделано при препроцессинге
  string = string.lower()
  if not comma:
    return re.findall(r'[\d]+|[\w]+', string)
  if comma:
    return re.findall(r'[\d]+|[\w]+|\,', string)

In [178]:
def tokens_to_string(tokens, string):
  pattern = r".?.?".join(tokens)
  found = re.search(pattern, string.lower())
  if found==None:
    split = len(string)
  else:
    split = found.start()
  return split

In [599]:
def clarify_address(tokens, types):
  # add missing "number" type
  for i, token in enumerate(tokens):
    if token.isdigit():
      types[i] = 'число'
      if i == 0:
        types[i] = "дом"
      elif types[i-1] in sep_house_signs and types[i-1] != 'литера' and types[i-1] != 'дробь':
        types[i] = types[i-1]
      elif i>=2 and types[i-1] == 'дробь':
        types[i] = types[i-2]
        types[i-1] = types[i-2]
    elif types[i] == 'литера' and i!=0:
      tokens[i-1] += tokens[i]
    
  # write this info somewhere
  dic = {}
  for token, typ in zip(tokens, types):
    if token not in house_signs_inv and typ != 'препинания':
      if typ not in dic:
        dic[typ] = token
        
  # rename
  dic['Дом'] = dic.get('дом', '') 
  if len(dic.get('корпус', '')) > len(dic.get('строение', '')):
    dic['Корпус/строение'] = dic['корпус']
  elif dic.get('строение', False):
    dic['Корпус/строение'] = dic['строение']
  return dic

In [600]:
def extract_house_tokens(tokens):
  a = lambda x: "число" if x.isdigit() and len(x)<6 else "препинания" if x == ',' else "не распознано"
  types = [house_signs_inv.get(x, a(x)) for x in tokens]
  types_bin = [0 if x=='не распознано' else 1  for x in types]
  array = list((list(y) for (x,y) in itertools.groupby((enumerate(types_bin)),operator.itemgetter(1)) if x == 1))
  if len(array) == 0:
    return [], []
  longest_seq = max(reversed(array), key=len)
  return [tokens[i] for (i, _) in longest_seq], [types[i] for (i, _) in longest_seq]

def extract_house(string): #from 2.0
  '''
  Обёртка для процедуры извлечения номера дома/корпуса от оставшейся строки
  Вход: адрес(строка)
  Выход: адрес без номеров дома/корпуса, номера дома/корпуса строкой
  '''
  tokens = tokenize(string, comma=True)
  house_tokens, house_types = extract_house_tokens(tokens)
  
  split = tokens_to_string(house_tokens, string)
  
  house = clarify_address(house_tokens, house_types)
  address = string[:split]
  # house = string[split:]
  return address, house

In [601]:
extract_house("улица Лобачика, д 4 а корпус 6")

('улица Лобачика',
 {'Дом': '4а', 'Корпус/строение': '6', 'дом': '4а', 'корпус': '6'})

In [674]:
stopwords = {
  'российская':'',
  'федерация':'',
  'орел':'орёл',
  'мо':'московская обл',
  'большой':"(б OR большой)",
  'большая':"(б OR большая)",
  'малый':"(м OR малый)",
  'малая':"(м OR малая)",
  'средний':'(ср OR с OR средний)',
  'средняя':'(ср OR с OR средняя)',
  'нижний':'(н OR нижний)',
  'б':"(б OR большая OR большой)",
  'с':'(ср OR с OR средняя OR средний)',
  'ср':'(ср OR с OR средняя OR средний)',
  'м':"(м OR малый OR малая)",
  'н':'(н OR нижний)',
  '/':''
}

In [711]:
def optimize_for_search(string):
  string = string.replace('ё', 'е')
  string = multiple_replace(stopwords, string.lower())
  string = multiple_replace(replaces_inv, string)
  #string = re.sub(r"[а-яА-Я]{4,}", '\g<0>~^2', string)
  return string

In [494]:
housenum_replaces = {
  '/':'\/'
}

In [502]:
def optimize_housenum(string):
  # creates escape characters for elasticsearch
  string = multiple_replace(housenum_replaces, string)
  return '"' + string+ '"'

In [459]:
re.sub(r"[а-яА-Я]{5,}", '\g<0>~', "г^0.5 москва зеленоград каштановая (аллея OR ал)^0.5 ")

'г^0.5 москва~ зеленоград~ каштановая~ (аллея~ OR ал)^0.5 '

In [460]:
optimize_for_search("г москва зеленоград каштановая аллея")

'г^0.5 москва~ зеленоград~ каштановая~ (аллея~ OR ал)^0.5'

In [461]:
re.sub(r"[а-яА-Я]{5,}", '\g<0>~', "российская федерация г^0.5 санкт петербург ириновский (проспект OR пр-кт)^0.5 ")

'российская~ федерация~ г^0.5 санкт~ петербург~ ириновский~ (проспект~ OR пр-кт)^0.5 '

In [732]:
def verify_address(full_address, debug=True):
  '''
  Ищет адрес в ФИАС
  Вход: строка
  Выход: словарь с полным адресом по ФИАС и его составляющими
  '''
  if full_address == '':
    return []
  string = optimize_for_search(full_address)
  query = {
  'size':1,
  "query":
    {"query_string" : 
      { "fields" : ["fullname"],
        "query" : string,
        "fuzziness":"auto",
        "use_dis_max" :  "true"
      }
    }
  }
  response = es.search(index='fias_full_text', doc_type='address', body=query)
  
#   # Найти номер дома в ФИАС
#   if dic.get('Улица', False):
#     response.update(verify_home(dic, response['guid']))
#   response.update({'original':dic['original']})
  if debug:
    dic = response["hits"]["hits"][0]["_source"]
    dic['query'] = string
    return dic

  
  try:
    return response["hits"]["hits"][0]["_source"]
  except IndexError:
    return []

In [721]:
def verify_home(dic, aoguid, index, debug=False):
  query = {
    "size":1,
    "query": {
      "bool": {
        "must":[],
        "should": [],  ## Here goes your stuff
        "must_not":[]
      }
    }
  }
  cases = query['query']['bool']['should']
  must =  query['query']['bool']['must']
  must_not =  query['query']['bool']['must_not']
  must.append({"match" : {
      "AOGUID": aoguid
    }})
  if dic.get('дом', False):
    must.append({"match" :{
      "HOUSENUM":optimize_housenum(dic["дом"])
    }})    
  if dic.get('корпус', False):
    cases.append({"match" :{
      "BUILDNUM":optimize_housenum(dic["корпус"])
    }})
  else:
    must_not.append({"match" : 
                     {"BUILDNUM":"*"} 
                    })
  if dic.get('строение', False):
    cases.append({"match" :{
      "STRUCNUM":optimize_housenum(dic['строение'])
    }})
  else:
    must_not.append({"match":
                     {"STRUCNUM":'*'}
                    })
  if index:
    cases.append({"match" :{
      "POSTALCODE":'"'+index+'"'
    }})
    
    
  must.append({"bool":{
    
  }})
    
  response = es.search(index='fias_houses', body=query)
  if len(response["hits"]["hits"])==0:
    dic.update({"комментарий":"дом не найден в ФИАС"})
    return dic
  else:
    response = response["hits"]["hits"][0]["_source"]
    
  if response["BUILDNUM"] == response["HOUSENUM"]:
    response['Корпус/строение'] = response["BUILDNUM"]
  elif len(response["BUILDNUM"])>len(response["HOUSENUM"]):
    response['Корпус/строение'] = response["BUILDNUM"]
  elif len(response["BUILDNUM"])>len(response["HOUSENUM"]):
    response['Корпус/строение'] = response["BUILDNUM"]
  else:
    response['Корпус/строение'] = response["BUILDNUM"]
  new_dic = {key.lower():response[key] for key in ["HOUSENUM", "BUILDNUM", "STRUCNUM", "POSTALCODE", "HOUSEID"]}
  new_dic.update({'house query':str(query)})
 
  return new_dic

In [715]:
def standardize(string, origin = True, debug=False):
  '''
  Обёртка для всех методов выше. Разделяет адрес на его составляющие и ищет совпадение в ФИАС. В 90+% случаев находит.
  Вход: строка с адресом
  Выход: составляющие адреса
  '''
  dic = {}
  if origin:
    dic['origin'] = string
    
  string = preprocess(string)
  address, index = extract_index(string)
  try:
    index = index.strip()
  except AttributeError:
    pass
  address, house = extract_house(address)
  dic['index']=index
  dic['address'] = address
  dic.update(verify_address(address))
  if dic.get('street', False):
    dic.update(verify_home(house, dic['guid'], index))
  dic.update(house) 
  return dic

In [784]:
standardize("142703, Московская область, Ленинский район, г.Видное, ул. Школьная, д.78", debug=True)

{'address': 'Московская область Ленинский район г Видное ул Школьная ',
 'aolevel': '7',
 'area': 'Ленинский',
 'area_type': 'р-н',
 'buildnum': '',
 'city': 'Видное',
 'city_type': 'г',
 'fullname': 'обл Московская, р-н Ленинский, г Видное, ул Школьная',
 'guid': 'b130660f-73d3-49cc-b685-de13d8d065ed',
 'house query': '{\'size\': 1, \'query\': {\'bool\': {\'must\': [{\'match\': {\'AOGUID\': \'b130660f-73d3-49cc-b685-de13d8d065ed\'}}, {\'match\': {\'HOUSENUM\': \'"78"\'}}], \'should\': [{\'match\': {\'POSTALCODE\': \'"142703"\'}}], \'must_not\': [{\'match\': {\'BUILDNUM\': \'*\'}}, {\'match\': {\'STRUCNUM\': \'*\'}}]}}}',
 'houseid': '672cb04a-f4d0-43a8-bcdb-605a9fbd76d7',
 'housenum': '78',
 'index': '142703',
 'origin': '142703, Московская область, Ленинский район, г.Видное, ул. Школьная, д.78',
 'postalcode': '142703',
 'query': 'московская обл ленинский р-н г видное ул школьная ',
 'region': 'Московская',
 'region_type': 'обл',
 'street': 'Школьная',
 'street_type': 'ул',
 'strucnu

In [734]:
standardize("129164, Российская Федерация, г. Москва, г. Москва, Ярославская ул, дом 8, корпус 3")

{'address': 'Российская Федерация г Москва г Москва Ярославская ул ',
 'aolevel': '7',
 'buildnum': '3',
 'fullname': 'г Москва, ул Ярославская',
 'guid': '9c8dba99-bb48-459f-bc5f-a73e9e43e7d0',
 'house query': '{\'size\': 1, \'query\': {\'bool\': {\'must\': [{\'match\': {\'AOGUID\': \'9c8dba99-bb48-459f-bc5f-a73e9e43e7d0\'}}, {\'match\': {\'HOUSENUM\': \'"8"\'}}], \'should\': [{\'match\': {\'BUILDNUM\': \'"3"\'}}, {\'match\': {\'POSTALCODE\': \'"129164"\'}}], \'must_not\': [{\'match\': {\'STRUCNUM\': \'*\'}}]}}}',
 'houseid': 'f694b364-9b36-40a9-8416-e53f933bbd44',
 'housenum': '8',
 'index': '129164',
 'origin': '129164, Российская Федерация, г. Москва, г. Москва, Ярославская ул, дом 8, корпус 3',
 'postalcode': '129164',
 'query': '  г москва г москва ярославская ул ',
 'region': 'Москва',
 'region_type': 'г',
 'street': 'Ярославская',
 'street_type': 'ул',
 'strucnum': '',
 'Дом': '8',
 'Корпус/строение': '3',
 'дом': '8',
 'корпус': '3'}

In [558]:
def get_addr(strings, progress=True):
  '''
  Обрабатывает несколько адресов подряд.
  Вход: массив строк
  Выход: pandas Dataframe
  '''
  dics = []
  n = len(strings) - 1
  for i,  line in enumerate(strings):
    if progress:
      print("Working on {0} of {1}. Progress {2:03.1f}%".format(i, n, (i/n)*100), end='\r')
    dics.append(standardize(line))
  return pd.DataFrame(dics)

In [739]:
def score(ref, orig_col="Исходный адрес", func = get_addr, cols_to_score = ["Регион", "Город", "Н/п", "Район", "Улица", "Дом", "Корпус/строение"]):
  df_1 = func(ref[orig_col]).rename(index=str, columns=dadata_LUT)
  if 'Индекс' in cols_to_score:
    ref["Индекс"] = ref["Индекс"].fillna('9999999').astype(int).astype(str).replace('9999999', '')
  ref, df_1 = ref.fillna(''), df_1.fillna('')
  df_1 = df_1.to_dict(orient='records')
  N = ref.shape[0]
  ref=ref.to_dict(orient='records')
  n = 0
  correct = 0
  df = []
  if len(df_1) != len(ref):
    return "не совпадают размеры таблицы"
  for i, row in enumerate(ref):
    for key, value in row.items():
      if key in cols_to_score:   #["Регион", "Индекс", "Район", "Город", "Н/п", "Улица", "Дом", "Корпус/строение"]:
        n += 1
        if str(value) == str(df_1[i][key]):
          correct += 1
        else:
          df.append(df_1[i])
  print("\n{0:03.1f}% correct fields".format(correct/n*100))
  df = pd.DataFrame(df).drop_duplicates()
  print("\n{0:03.1f}% correct lines".format((N-df.shape[0])/N*100))
  return df

In [782]:
def score_by_id(ref, orig_col="Исходный адрес", func = get_addr):
  ref['Тип корпуса/строения'].astype(str)
  ref = ref[[orig_col, 'Код ФИАС']][(ref['Уровень по ФИАС']=='8: дом') & (ref['Тип корпуса/строения'].astype(str) != 'nan')]
  predicted_df = func(ref[orig_col])
  true_id = ref['Код ФИАС']
  predicted_id = predicted_df['houseid']
  correct = 0
  incorrect = 0
  for true, predicted in zip(true_id, predicted_id):
    if true == predicted:
      correct += 1
    else:
      incorrect += 1
  accuracy = correct/(correct+incorrect)*100
  print()
  print("Accuracy: {0:03.3f}%".format(accuracy))

In [750]:
dadata_LUT = {'original':'Исходный адрес',
  'fullname':'Адрес',
  'index':'Индекс',
  'postalcode':'Индекс',
  'country':'Страна',
  'region_type':'Тип региона',
  'region':'Регион',
  'area_type':'Тип района',
  'area':'Район',
  'city_type':'Тип города',
  'city':'Город',
  '65_type':'Тип н/п',
  '65':'Н/п',
  '???':'Адм. округ',
  'town':'Н/п',
  'town_type':'Тип н/п',
  'district_type':'Тип района',
  'district':'Район города',
  'street_type':'Тип улицы',
  'street':'Улица',
  'house_type':'Тип дома',
#   'housenum':'Дом',
#   'build_type':'Тип корпуса/строения',
#   'buildnum':'Корпус/строение',
#   'struc_type':'Тип корпуса/строения',
#   'strucnum':'Корпус/строение',            
  'flat_type':'Тип квартиры',
  'flat_num':'Номер Квартиры',
  'houseid':"Код ФИАС"
}

In [783]:
score_by_id(ref)

Working on 362 of 362. Progress 100.0%
Accuracy: 82.369%


In [755]:
ref.head()

,Исходный адрес,Адрес,Индекс,Страна,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Адм. округ,Район города,Тип улицы,Улица,Тип дома,Дом,Тип корпуса/строения,Корпус/строение,Тип квартиры,Квартира,Абонентский ящик,Код КЛАДР,Код ФИАС,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС для физических лиц,Площадь квартиры,Стоимость м²,Стоимость квартиры,Часовой пояс,Широта,Долгота,Внутри кольцевой?,Расстояние от кольцевой,Точность координат,Подходит для рассылки?,Код проверки,Нераспознанная часть
0,"142784, г.Москва, г.Московский, 1-й мкр., д.35...","г Москва, г Московский, мкр 1-й, д 35, оф 4",108811.0,Россия,г,Москва,NaN,NaN,г,Московский,NaN,NaN,Новомосковский,Новомосковский,мкр,1-й,д,35,NaN,NaN,оф,4,NaN,7700000600000029696,c3821934-8a9e-4412-9d58-b11110cea32a,8: дом,0,45297565001,4.595200e+07,7751,NaN,NaN,NaN,UTC+3,55.602106,37.352028,За МКАД,8.0,Улица,Да,Корректный,NaN
1,"142703, Московская область, Ленинский район, г...","Московская обл, г Видное, ул Школьная, д 78",142703.0,Россия,обл,Московская,р-н,Ленинский,г,Видное,NaN,NaN,NaN,NaN,ул,Школьная,д,78,NaN,NaN,NaN,NaN,NaN,5001400100000629760,672cb04a-f4d0-43a8-bcdb-605a9fbd76d7,8: дом,1,46228501000,4.662810e+10,5003,NaN,63881.0,NaN,UTC+3,55.554277,37.690089,За МКАД,3.0,Точные,Под вопросом: нет квартиры,Корректный,NaN
2,"111020 г. Москва, Юрьевский пер. д. 20 кв. 15","г Москва, пер Юрьевский, д 20, кв 15",111020.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Юго-восточный,Лефортово,пер,Юрьевский,д,20,NaN,NaN,кв,15,NaN,7700000000002599936,8894c269-e96e-4cdf-b367-6a356ae2d117,8: дом,0,45290564000,4.538800e+07,7722,44.2,183210.0,8097882.0,UTC+3,55.764193,37.717790,Внутри МКАД,NaN,Точные,Да,Корректный,NaN
3,"109052, Москва г, ул.Рязанский проспект, д.2","г Москва, пр-кт Рязанский, д 2",109052.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,пр-кт,Рязанский,д,2,NaN,NaN,NaN,NaN,NaN,77000000000007600,36b4d93f-38cd-4b02-9d0d-c005f14e38eb,7: улица,0,45000000000,NaN,7721,NaN,NaN,NaN,UTC+3,55.725871,37.759232,Внутри МКАД,NaN,Точные,Под вопросом: дома нет в ФИАС,Корректный,NaN
4,"105066, г. Москва Денисовский переулок, д.9","г Москва, пер Денисовский, д 9",105066.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Центральный,Басманный,пер,Денисовский,д,9,NaN,NaN,NaN,NaN,NaN,7700000000011799552,dcf85160-13c2-4a04-a19a-853092bc60c1,8: дом,0,45286555000,4.537500e+07,7701,NaN,243235.0,NaN,UTC+3,55.766577,37.672331,Внутри МКАД,NaN,Ближайший дом,Под вопросом: нет квартиры,Корректный,NaN


In [737]:
score(ref.sample(400))

Working on 399 of 399. Progress 100.0%
94.2% correct fields

70.8% correct lines


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


,address,aolevel,buildnum,guid,house query,houseid,housenum,origin,query,strucnum,Адрес,Город,Дом,Индекс,Корпус/строение,Н/п,Район,Регион,Тип города,Тип н/п,Тип района,Тип региона,Тип улицы,Улица,дом,квартира,комната,корпус,офис,помещение,прочее,строение
87,Московская область Подольский район п Кузнечик...,7,3,0b844cc4-8f46-48d1-8795-af1b7d628db1,"{'size': 1, 'query': {'bool': {'must': [{'matc...",7c98c448-793b-4acd-9ab4-28ad495efe78,8а,"142133, Московская область, Подольский район, ...",московская обл подольский р-н п кузнечики ул о...,,"обл Московская, г Подольск, п Кузнечики, ул Ок...",Подольск,8а,142133,,Кузнечики,,Московская,г,п,,обл,ул,Октябрьский мост,8а,,,,,,,
138,г Санкт Петербург В О Малый пр,7,,29167695-b47f-4103-b499-2c9cc5925be1,"{'size': 1, 'query': {'bool': {'must': [{'matc...",9cb457ed-9bd5-47e9-a0ee-a40f7d8b9b5f,46,"г. Санкт-Петербург, 199178, В.О., Малый пр., д...",г санкт петербург в о (м OR малый) (проспект O...,,"г Санкт-Петербург, пр-кт Малый П.С.",,46,690001,,,,Санкт-Петербург,,,,г,пр-кт,Малый П.С.,46,,,,,3,лита,
140,Российская Федерация г Москва ул Большая Почто...,7,,b836feea-e291-4a9a-8efc-8ab51c3fd12c,"{'size': 1, 'query': {'bool': {'must': [{'matc...",353afa1d-c0ba-4e6d-a52a-eb6e9f0c4581,55/59,"Российская Федерация, 105082, г. Москва, ул. Б...",г москва ул (б OR большая) почтовая,1,"г Москва, ул Почтовая Б.",,55,105082,1,,,Москва,,,,г,ул,Почтовая Б.,55,,,,,,,1
116,г Москва ленинский проспект,7,,5f2a1243-a57b-418e-baee-ff76f4993b45,"{'size': 1, 'query': {'bool': {'must': [{'matc...",aeafa889-69ec-41d5-b590-2995107c1065,2А,"119991, г. Москва, ленинский проспект, д. 2А, ...",г москва ленинский (проспект OR пр-кт)^0.5,,"г Москва, пр-кт Ленинский",,2а,119049,,,,Москва,,,,г,пр-кт,Ленинский,2а,,,,1307,,,
6,Российская Федерация г Ярославль ул Республика...,7,,f42bc570-28b7-4167-a72d-582d84f2b6bd,"{'size': 1, 'query': {'bool': {'must': [{'matc...",c104dc3c-879d-430f-91b6-834f7381f5a0,42/24,"Российская Федерация, 150040, г. Ярославль, ул...",г ярославль ул республиканская,,"обл Ярославская, г Ярославль, ул Республиканская",Ярославль,42,150040,,,,Ярославская,г,,,обл,ул,Республиканская,42,,,,,,,
52,г Москва Большой Златоустинский пер,7,,6d9b6b8f-3b7b-4a50-b30a-63c457cfa502,"{'size': 1, 'query': {'bool': {'must': [{'matc...",ae8c4ea0-b90c-45e3-b460-45f46342e7fb,6,"101000, г. Москва, Большой Златоустинский пер....",г москва (б OR большой) златоустинский пер,2,"г Москва, пер Златоустинский Б.",,6,101000,стрб,,,Москва,,,,г,пер,Златоустинский Б.,6,,,,,,,стрб
15,г Москва ул Нижняя Красносельская,7,,4ba705ed-7ff3-4cab-96d7-22895261bd76,"{'size': 1, 'query': {'bool': {'must': [{'matc...",cf6255cf-c355-4be8-aeb5-344cb0578ebb,35,"105066, г. Москва, ул. Нижняя Красносельская, ...",г москва ул нижняя красносельская,,"г Москва, ул Красносельская М.",,35,187435,1а,,,Москва,,,,г,ул,Красносельская М.,35,,,,,,,1а
114,Россия г Пермь Шоссе Космонавтов,65,,a40098a2-f905-43c0-adbd-23a53871eba2,,,,"Россия, 614990, г. Пермь, Шоссе Космонавтов, д...",россия г пермь шоссе космонавтов,,"край Пермский, г Пермь, тер ГСК 145 Шоссе Косм...",Пермь,115,,,,,Пермский,г,,,край,,,115,,,,,,,
83,Москва ул Нижняя Первомайская,7,,d5c1834f-adc0-45da-9ac5-5be422396d18,"{'size': 1, 'query': {'bool': {'must': [{'matc...",8c16b501-8823-470a-a33d-e32a33b1c7bb,44,"105203,Москва, ул.Нижняя Первомайская,д.44",москва ул нижняя первомайская,,"г Москва, ул Нижняя",,44,658540,,,,Москва,,,,г,ул,Нижняя,44,,,,,,,
113,Российская Федерация г Москва г Москва Большая...,7,,683d7f9d-40ea-4bf1-b3c3-ff2eb628a5f7,"{'size': 1, 'query': {'bool': {'must': [{'matc...",61066a66-4a2b-4894-8a03-ffc718eef4d2,7а,"127256, Российская Федерация, г. Москва, г. Мо...",г москва г москва (б OR большая) марфинская,,"г Москва, ул Марфинская Б.",,7а,174210,,,,Москва,,,,г,ул,Марфинская Б.,7а,,,,,,,


In [745]:
score(ref2).to_excel("results/mistakes_ref2.xlsx", columns=order)

Working on 1040 of 1040. Progress 100.0%
92.7% correct fields

72.9% correct lines


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  
/usr/local/lib/python3.6/site-packages/pandas/io/formats/excel.py:372: FutureWarning: Not all names specified in 'columns' are found; this will raise a KeyError in the future
  FutureWarning)


In [746]:
score(ref).to_excel("results/mistakes_ref1.xlsx", columns=order)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  
/usr/local/lib/python3.6/site-packages/pandas/io/formats/excel.py:372: FutureWarning: Not all names specified in 'columns' are found; this will raise a KeyError in the future
  FutureWarning)



94.9% correct fields

73.3% correct lines


In [744]:
order = ['origin', "Адрес", 'fullname', 'Дом', 'Корпус/строение', 'Индекс', 'Комментарий', 'query', 'house query']

In [742]:
pd.DataFrame(get_addr(ref['Исходный адрес']), columns=order).to_excel('results/27.02.xlsx')

KeyboardInterrupt: 

In [ ]:
Out[728].to_excel()

In [749]:
get_addr(ref['Исходный адрес'].sample(20))

,address,aolevel,area,area_type,buildnum,city,city_type,fullname,guid,house query,houseid,housenum,index,origin,postalcode,query,region,region_type,street,street_type,strucnum,town,town_type,Дом,Корпус/строение,дом,корпус,офис,строение
0,Московская обл г Солнечногорск ул Красноармейс...,7,Солнечногорский,р-н,,Солнечногорск,г,"обл Московская, р-н Солнечногорский, г Солнечн...",18427cf9-d264-4842-ae91-78ddcbfcb4ea,"{'size': 1, 'query': {'bool': {'must': [{'matc...",9e9ce13d-4bae-4ad6-b25d-e1082f211576,8,141506,"141506, Московская обл. г.Солнечногорск, ул. К...",368027,московская обл г солнечногорск ул красноармейс...,Московская,обл,Красноармейская,ул,,NaN,NaN,8,NaN,8,NaN,99,NaN
1,Россия г Москва г Троицк ул Лесная,7,NaN,NaN,,Троицк,г,"г Москва, г Троицк, ул Лесная",96c3ee2e-9f13-4065-868d-4af812fae194,"{'size': 1, 'query': {'bool': {'must': [{'matc...",7b13d02e-ccbc-4b26-a5b6-f7de85a4440b,8А,142190,"142190,Россия, г. Москва, г. Троицк, ул. Лесна...",393670,россия г москва г троицк ул лесная,Москва,г,Лесная,ул,,NaN,NaN,8а,NaN,8а,NaN,NaN,NaN
2,г Москва ул Профсоюзная,7,NaN,NaN,4,NaN,NaN,"г Москва, ул Профсоюзная",9a661426-e65c-4223-af4d-aa5e95b7caac,"{'size': 1, 'query': {'bool': {'must': [{'matc...",7d483159-0c6b-4aca-9449-b641179f9035,140,117321,"117321, г. Москва, ул. Профсоюзная, д. 140, к. 4",117321,г москва ул профсоюзная,Москва,г,Профсоюзная,ул,,NaN,NaN,140,4,140,4,NaN,NaN
3,Московская область г Солнечногорск ул Красноар...,7,Солнечногорский,р-н,,Солнечногорск,г,"обл Московская, р-н Солнечногорский, г Солнечн...",18427cf9-d264-4842-ae91-78ddcbfcb4ea,"{'size': 1, 'query': {'bool': {'must': [{'matc...",9e9ce13d-4bae-4ad6-b25d-e1082f211576,8,141506,"141506, Московская область, г.Солнечногорск, у...",368027,московская обл г солнечногорск ул красноармейс...,Московская,обл,Красноармейская,ул,,NaN,NaN,8,NaN,8,NaN,99,NaN
4,Российская Федерация Москва Дубининская ул,7,NaN,NaN,2,NaN,NaN,"г Москва, ул Дубининская",2397f14a-c14f-4ddd-a30f-7223a3342d6c,"{'size': 1, 'query': {'bool': {'must': [{'matc...",30ec791e-bd70-4f79-b237-0799fdb2213a,59,115054,"115054,Российская Федерация, Москва, Дубининск...",115054,москва дубининская ул,Москва,г,Дубининская,ул,1,NaN,NaN,59,1,59,2,NaN,1
5,Тюменская обл Тюменский р н Патрушева д ул Дру...,7,Тюменский,р-н,,NaN,NaN,"обл Тюменская, р-н Тюменский, д Патрушева, ул ...",ddbe3566-1bb6-486a-aec2-e6d0bd9264cc,"{'size': 1, 'query': {'bool': {'must': [{'matc...",a94db2ba-06df-4e7b-9c69-eb95a8343d82,6,625519,"625519, Тюменская обл, Тюменский р-н, Патрушев...",625519,тюменская обл тюменский р-н (н OR нижний) патр...,Тюменская,обл,Дружбы,ул,,Патрушева,д,6,NaN,6,NaN,NaN,NaN
6,Российская Федерация г Москва ул Б Новодмитров...,7,NaN,NaN,,NaN,NaN,"г Москва, ул Новодмитровская Б.",e0907613-4e2e-416b-a7e0-43cd9bfeb235,"{'size': 1, 'query': {'bool': {'must': [{'matc...",c0e80f57-a5bc-4c5c-9313-58270bc7a67d,14,127015,"127015, Российская Федерация, г. Москва, ул. Б...",127015,г москва ул (б OR большая OR большой) новодм...,Москва,г,Новодмитровская Б.,ул,1,NaN,NaN,14,1,14,NaN,NaN,1
7,Российская Федерация 77 Москва г Москва Садовн...,7,NaN,NaN,,NaN,NaN,"г Москва, ул Садовническая",b0eb036a-e240-4dc3-9dfb-7b01d3ed8aa5,"{'size': 1, 'query': {'bool': {'must': [{'matc...",04f84a76-6e1b-4c38-8d66-03a19fb35489,42,115035,"115035, Российская Федерация, 77 Москва, г. Мо...",115035,77 москва г москва садовническая ул,Москва,г,Садовническая,ул,4,NaN,NaN,42,4,42,NaN,NaN,4
8,г Москва Волгоградский пр,7,NaN,NaN,5,NaN,NaN,"г Москва, пр-кт Волгоградский",3fbbba41-86cb-4779-9631-e96dd73f536a,"{'size': 1, 'query': {'bool': {'must': [{'matc...",f58be6ad-08a6-4d11-a608-42d68b601e16,113,109117,"109117, г. Москва, Волгоградский пр.,д.113,к.5",109117,г москва волгоградский (проспект OR пр-кт)^0.5,Москва,г,Волгоградский,пр-кт,,NaN,NaN,113,5,113,5,NaN,NaN
9,г Москва Старомонетный пер,7,NaN,NaN,,NaN,NaN,"г Москва, пер Старомонетный",8c0d4520-4057-4de3-ae6b-80b7e177be5c,"{'size': 1, 'query': {'bool': {'must': [{'matc...",aaab873d-835e-4029-850f-f0

In [758]:
get_addr(ref["Исходный адрес"].head())

,address,aolevel,area,area_type,buildnum,city,city_type,fullname,guid,house query,houseid,housenum,index,origin,postalcode,query,region,region_type,street,street_type,strucnum,Дом,дом,квартира,офис
0,г Москва г Московский 1 й мкр,7,NaN,NaN,,Московский,г,"г Москва, г Московский, мкр 1-й",80f7c7f5-dfc2-4c1a-ba9e-42ec4b67a5ac,"{'size': 1, 'query': {'bool': {'must': [{'matc...",951673df-57bc-44af-b0bf-9e614594df3a,35,142784,"142784, г.Москва, г.Московский, 1-й мкр., д.35...",142784,г москва г московский 1 й мкр,Москва,г,1-й,мкр,,35,35,NaN,4
1,Московская область Ленинский район г Видное ул...,7,Ленинский,р-н,,Видное,г,"обл Московская, р-н Ленинский, г Видное, ул Шк...",b130660f-73d3-49cc-b685-de13d8d065ed,"{'size': 1, 'query': {'bool': {'must': [{'matc...",672cb04a-f4d0-43a8-bcdb-605a9fbd76d7,78,142703,"142703, Московская область, Ленинский район, г...",142703,московская обл ленинский р-н г видное ул школь...,Московская,обл,Школьная,ул,,78,78,NaN,NaN
2,г Москва Юрьевский пер,7,NaN,NaN,,NaN,NaN,"г Москва, пер Юрьевский",81b15723-26cb-482d-997a-f66f80296d54,"{'size': 1, 'query': {'bool': {'must': [{'matc...",8894c269-e96e-4cdf-b367-6a356ae2d117,20,111020,"111020 г. Москва, Юрьевский пер. д. 20 кв. 15",111020,г москва юрьевский пер,Москва,г,Юрьевский,пер,,20,20,15,NaN
3,Москва г ул Рязанский проспект,7,NaN,NaN,,NaN,NaN,"г Москва, пр-кт Рязанский",36b4d93f-38cd-4b02-9d0d-c005f14e38eb,"{'size': 1, 'query': {'bool': {'must': [{'matc...",1dc92fb4-4856-4f92-966c-2cd077a990aa,2,109052,"109052, Москва г, ул.Рязанский проспект, д.2",109052,москва г ул рязанский (проспект OR пр-кт)^0.5,Москва,г,Рязанский,пр-кт,75,2,2,NaN,NaN
4,г Москва Денисовский переулок,7,NaN,NaN,,NaN,NaN,"г Москва, пер Денисовский",0b1591b1-6a25-4d35-8f21-38be9d7b29cd,"{'size': 1, 'query': {'bool': {'must': [{'matc...",dcf85160-13c2-4a04-a19a-853092bc60c1,9,105066,"105066, г. Москва Денисовский переулок, д.9",105066,г москва денисовский пер,Москва,г,Денисовский,пер,,9,9,NaN,NaN


In [66]:
house_signs_inv

{'а': 'литера',
 'б': 'литера',
 'в': 'литера',
 'вл': 'владение',
 'владение': 'владение',
 'д': 'дом',
 'дом': 'дом',
 'е': 'литера',
 'ж': 'литера',
 'з': 'литера',
 'к': 'корпус',
 'к-т': 'кабинет',
 'каб': 'кабинет',
 'каб-т': 'кабинет',
 'кабинет': 'кабинет',
 'кв': 'квартира',
 'квартира': 'квартира',
 'ком': 'комната',
 'комн': 'комната',
 'комната': 'комната',
 'копр': 'корпус',
 'кор': 'корпус',
 'корп': 'корпус',
 'корпус': 'корпус',
 'лит': 'прочее',
 'литер': 'прочее',
 'литера': 'прочее',
 'оф': 'офис',
 'офис': 'офис',
 'пом': 'помещение',
 'помещение': 'помещение',
 'с': 'строение',
 'стр': 'строение',
 'строен': 'строение',
 'строение': 'строение'}

In [67]:
def load_types():
  sep_region_signs = {
    'обл.': {"область", "обл", "обл-ть", 'МО'},
    'респ.': {"республика", 'респ'},
    'край': {'край'},
    'город': {'г', 'гор', 'город'},
    'АО':{'ао', 'автономный', 'округ', 'автономная'}
  }

  sep_street_signs = {
    'аллея':{'аллея', 'а'},
    'бульвар':{'б-р', 'бульвар', 'б'}, #- надо добавлять special case
    'наб.':{'наб', 'набережная'},
    'переулок':{'пер', 'переулок'},
    'площадь':{'пл', "площадь"},
    "проспект":{"пр", "пр-кт", "просп"},
    "проезд":{"проезд", "пр-д"},
    "ул.":{"улица", "ул", "у", 'ул-ца'}
  }

  sep_district_signs = {
    'р-н':{'район', "р" , "р-н"},
    'п':{'п', 'пгт', 'поселок', "посёлок", "городского", "типа"}
  }

  sep_city_signs  = {
    'город':{'г', 'гор', 'город'},
    'село':{'с', 'село', 'сел'},
    'деревня':{'д', 'дер', 'деревня', 'д-ня'},
    'с/п':{"сельский", "поселок", "сп"}
  }

  sep_house_signs = {
    'дом': {'вл', 'д', 'дом'},
    'корпус': {'к', 'корп', 'копр', 'кор', 'корпус', 'с', 'стр', 'строен', 'строение'},
    'квартира': {'кабинет', 'кв', 'квартира', "ком",'комн', 'пом', 'помещение', 'комната'},
    'офис': {'оф', 'офис'},
    'литера': {'а', 'б', 'в'}
  }
  
  types = {
    'город':sep_city_signs,
    'дом':sep_house_signs,
    'улица':sep_street_signs,
    'район':sep_district_signs,
    'регион':sep_region_signs,
    'страна':{'страна', 'стр'},
    'индекс':{'индекс', 'инд'}
  }
  
  return types

In [70]:
dic

{'город': {'город': {'г', 'гор', 'город'},
  'деревня': {'д', 'д-ня', 'дер', 'деревня'},
  'с/п': {'поселок', 'сельский', 'сп'},
  'село': {'с', 'сел', 'село'}},
 'дом': {'дом': {'вл', 'д', 'дом'},
  'квартира': {'кабинет',
   'кв',
   'квартира',
   'ком',
   'комн',
   'комната',
   'пом',
   'помещение'},
  'корпус': {'к',
   'копр',
   'кор',
   'корп',
   'корпус',
   'с',
   'стр',
   'строен',
   'строение'},
  'литера': {'а', 'б', 'в'},
  'офис': {'оф', 'офис'}},
 'индекс': {'инд', 'индекс'},
 'район': {'п': {'городского', 'п', 'пгт', 'поселок', 'посёлок', 'типа'},
  'р-н': {'р', 'р-н', 'район'}},
 'регион': {'АО': {'автономная', 'автономный', 'ао', 'округ'},
  'город': {'г', 'гор', 'город'},
  'край': {'край'},
  'обл.': {'МО', 'обл', 'обл-ть', 'область'},
  'респ.': {'респ', 'республика'}},
 'страна': {'стр', 'страна'},
 'улица': {'аллея': {'а', 'аллея'},
  'бульвар': {'б', 'б-р', 'бульвар'},
  'наб.': {'наб', 'набережная'},
  'переулок': {'пер', 'переулок'},
  'площадь': {'п

In [388]:
replaces_inv = inverdic(replaces_lowered)

In [164]:
multiple_replace(replaces_inv, "352380 , КРОПОТКИН, сельский поселок танковый р-н, ЛЕСНОЙ ПЕРЕУЛОК, главная аллея, д. 20 5".lower())

'352380 , кропоткин, с/п танковый р-н, лесной пер, главная (аллея OR ал), д. 20 5'

In [159]:
re.sub(re.escape(r'\bпос(е|ё)лок\b'), r"пос",'КРОПОТКИН, сельский поселок танковый ')

'КРОПОТКИН, сельский поселок танковый '

'\\\\b\\п\\о\\с\\(\\е\\|\\ё\\)\\л\\о\\к\\\\b'

In [380]:
replaces_inv

{'МО': 'обл московская',
 'а': '(аллея OR ал)',
 'а обл': '(а.обл OR аобл)',
 'а.окр': '(ао OR а.окр)',
 'авт.обл': '(а.обл OR аобл)',
 'автономная область': '(а.обл OR аобл)',
 'автономный': '(ао OR а.окр)',
 'автономный округ': '(ао OR а.окр)',
 'ал': '(аллея OR ал)',
 'аллея': '(аллея OR ал)',
 'аобл': '(а.обл OR аобл)',
 'аокр': '(ао OR а.окр)',
 'б-р': 'б-р',
 'бульвар': 'б-р',
 'г': 'г',
 'гор': 'г',
 'город': 'г',
 'д': 'д',
 'д-ня': 'д',
 'дер': 'д',
 'деревня': 'д',
 'край': 'край',
 'наб': 'наб',
 'набережная': 'наб',
 'обл': 'обл',
 'обл-ть': 'обл',
 'область': 'обл',
 'пер': 'пер',
 'переулок': 'пер',
 'пл': '(площадь OR пл)',
 'площадь': '(площадь OR пл)',
 'пос': 'п',
 'пос гор типа': 'пгт',
 'пос. гор. типа': 'пгт',
 'пос.гор.типа': 'пгт',
 'поселок': 'п',
 'поселок городского типа': 'пгт',
 'посёлок': 'п',
 'посёлок городского типа': 'пгт',
 'пр': '(проспект OR пр-кт)',
 'пр-д': 'пр-д',
 'пр-кт': '(проспект OR пр-кт)',
 'прд': 'пр-д',
 'проезд': 'пр-д',
 'просп': '(прос

In [124]:
'переулок' == 'переулок'

True

In [754]:
ref = pd.read_excel('ref/references.xlsx', sheet_name='Good')

In [181]:
ref

,Исходный адрес,Адрес,Индекс,Страна,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Адм. округ,Район города,Тип улицы,Улица,Тип дома,Дом,Тип корпуса/строения,Корпус/строение,Тип квартиры,Номер Квартиры,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,"142784, г.Москва, г.Московский, 1-й мкр., д.35...","г Москва, г Московский, мкр 1-й, д 35, оф 4",108811.0,Россия,г,Москва,NaN,NaN,г,Московский,NaN,NaN,Новомосковский,Новомосковский,мкр,1-й,д,35,NaN,NaN,оф,4,NaN,7700000600000029696,c3821934-8a9e-4412-9d58-b11110cea32a,8: дом,0,45297565001,4.595200e+07,7751,NaN,NaN,NaN,UTC+3,55.602106,37.352028,За МКАД,8.0,Улица,Да,Корректный
1,"142703, Московская область, Ленинский район, г...","Московская обл, г Видное, ул Школьная, д 78",142703.0,Россия,обл,Московская,р-н,Ленинский,г,Видное,NaN,NaN,NaN,NaN,ул,Школьная,д,78,NaN,NaN,NaN,NaN,NaN,5001400100000629760,672cb04a-f4d0-43a8-bcdb-605a9fbd76d7,8: дом,1,46228501000,4.662810e+10,5003,NaN,63881.0,NaN,UTC+3,55.554277,37.690089,За МКАД,3.0,Точные,Под вопросом: нет квартиры,Корректный
2,"111020 г. Москва, Юрьевский пер. д. 20 кв. 15","г Москва, пер Юрьевский, д 20, кв 15",111020.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Юго-восточный,Лефортово,пер,Юрьевский,д,20,NaN,NaN,кв,15,NaN,7700000000002599936,8894c269-e96e-4cdf-b367-6a356ae2d117,8: дом,0,45290564000,4.538800e+07,7722,44.2,183210.0,8097882.0,UTC+3,55.764193,37.717790,Внутри МКАД,NaN,Точные,Да,Корректный
3,"109052, Москва г, ул.Рязанский проспект, д.2","г Москва, пр-кт Рязанский, д 2",109052.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,пр-кт,Рязанский,д,2,NaN,NaN,NaN,NaN,NaN,77000000000007600,36b4d93f-38cd-4b02-9d0d-c005f14e38eb,7: улица,0,45000000000,NaN,7721,NaN,NaN,NaN,UTC+3,55.725871,37.759232,Внутри МКАД,NaN,Точные,Под вопросом: дома нет в ФИАС,Корректный
4,"105066, г. Москва Денисовский переулок, д.9","г Москва, пер Денисовский, д 9",105066.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Центральный,Басманный,пер,Денисовский,д,9,NaN,NaN,NaN,NaN,NaN,7700000000011799552,dcf85160-13c2-4a04-a19a-853092bc60c1,8: дом,0,45286555000,4.537500e+07,7701,NaN,243235.0,NaN,UTC+3,55.766577,37.672331,Внутри МКАД,NaN,Ближайший дом,Под вопросом: нет квартиры,Корректный
5,"105066, г. Москва, Денисовский переулок, д. 9","г Москва, пер Денисовский, д 9",105066.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Центральный,Басманный,пер,Денисовский,д,9,NaN,NaN,NaN,NaN,NaN,7700000000011799552,dcf85160-13c2-4a04-a19a-853092bc60c1,8: дом,0,45286555000,4.537500e+07,7701,NaN,243235.0,NaN,UTC+3,55.766577,37.672331,Внутри МКАД,NaN,Ближайший дом,Под вопросом: нет квартиры,Корректный
6,"140054, Московская обл., г. Котельники, ул. Ку...","Московская обл, г Котельники, ул Кузьминская, ...",140055.0,Россия,обл,Московская,NaN,NaN,г,Котельники,NaN,NaN,NaN,NaN,ул,Кузьминская,д,15,NaN,NaN,пом,11,NaN,5000003200000209920,2d71f8f4-7992-463f-9b87-8bac6a778218,8: дом,0,46444000000,4.673900e+07,5027,NaN,123623.0,NaN,UTC+3,55.667007,37.862058,За МКАД,2.0,Точные,Да,Корректный
7,"105318, г. Москва, ул. Ибрагимова, д.35, стр. ...","г Москва, ул Ибрагимова, д 35 стр 2, пом 1 комн 1",105318.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Восточный,Преображенское,ул,Ибрагимова,д,35,стр,2,пом,1 комн 1,NaN,7700000000013609984,1c1ddec3-932b-4b64-bd2e-baccc265cd8b,8: дом,0,45263594000,4.531600e+07,7719,NaN,NaN,NaN,UTC+3,55.789556,37.728174,Внутри МКАД,NaN,Точные,Да,Корректный
8,"127018, г. Москва. ул.Складочная, д.1А. стр.1","г Москва, ул Складочная, д 1А стр 1",127018.0,Россия,г,Москва,NaN,NaN,NaN,NaN,NaN,NaN,Северо-восточный,Бутырский,ул,Складочная,д,1А,стр,1,NaN,NaN,NaN,7700000000026619904,d6687618-aeab-49b6-88e7-fd87a2124cf6,8: дом,0,45280561000,4.535300e+07,7715,NaN,NaN,NaN,UTC+3,55.798797,37.595398,Внутри МКАД,NaN,Ближайший дом,Под вопросом: нет квартиры,Корректный
9,"119526, г. Москва, Вернадского проспе

In [271]:
house_signs_inv

{'а': 'литера',
 'б': 'литера',
 'в': 'литера',
 'вл': 'владение',
 'владение': 'владение',
 'д': 'дом',
 'дом': 'дом',
 'е': 'литера',
 'ж': 'литера',
 'з': 'литера',
 'к': 'корпус',
 'к-т': 'кабинет',
 'каб': 'кабинет',
 'каб-т': 'кабинет',
 'кабинет': 'кабинет',
 'кв': 'квартира',
 'квартира': 'квартира',
 'ком': 'комната',
 'комн': 'комната',
 'комната': 'комната',
 'копр': 'корпус',
 'кор': 'корпус',
 'корп': 'корпус',
 'корпус': 'корпус',
 'лит': 'прочее',
 'литер': 'прочее',
 'литера': 'прочее',
 'оф': 'офис',
 'офис': 'офис',
 'пом': 'помещение',
 'помещение': 'помещение',
 'с': 'строение',
 'стр': 'строение',
 'строен': 'строение',
 'строение': 'строение'}